In [ ]:
from __future__ import annotations

import copy
import dataclasses
from dataclasses import dataclass
from typing import List, TypeVar, Any, Tuple, Optional, Dict, Union
import pandas as pd
from typing import Generic
from typing import Literal

from autora.variable import VariableCollection
from sklearn.base import BaseEstimator

## First attempt – a dataframe wrapper which holds deltas

In [ ]:
class DataFrameWithDeltas:
    deltas: List[Tuple[str, pd.DataFrame]]

    def __init__(self, initial: pd.DataFrame):
        self.deltas = [("new", initial)]

    def update(self, delta: pd.DataFrame, kind: Literal["new", "extend"]):
        self.deltas.append((kind, delta))

    @property
    def df(self):
        v = None
        for kind, delta in self.deltas:
            if kind == "new":
                v = delta
            elif kind == "extend":
                assert isinstance(v, pd.DataFrame)
                v = pd.concat([v, delta])
            else:
                raise NotImplementedError(f"{kind=}")
        return v


In [ ]:
d = DataFrameWithDeltas(pd.DataFrame({"a": [1,2,3], "b": list("abc")}))
d.df

,a,b
0,1,a
1,2,b
2,3,c


In [ ]:
d.update(pd.DataFrame({"a": [4], "b": list("d")}), kind="extend")
d.df


,a,b
0,1,a
1,2,b
2,3,c
0,4,d


In [ ]:
d.update(pd.DataFrame({"a": [5], "b": list("e")}), kind="new")
d.df

,a,b
0,5,e


In [ ]:
d.deltas

[('new',
     a  b
  0  1  a
  1  2  b
  2  3  c),
 ('extend',
     a  b
  0  4  d),
 ('new',
     a  b
  0  5  e)]

## State with Deltas

literature_search / seed_data -> {conditions: […], observations: […], experimental_data: [...] model: […], status:
“expected”,
variables:
VariableCollection(…), kind = "new", source: "seed"}

experimentalist -> {conditions: […], status: “proposed”, kind="extend", source: "experimentalist"} ## the
experiment_runner only wants the "last" proposed conditions, so we need to be able to access the "last proposed
experiment"

experiment_runner -> {experimental_data: [...], conditions: view(data), observations: view(data), status:
“observed”, kind: "extend", source: "experiment_runner"}

theorist -> {model: TheModel(), (fitted_)data: [...], status: “fitted”, kind: "extend", source: "theorist"}

can "kind" just be the "extension" function rather than the abstract name.

Idea: we store all these results as "deltas":
all_data = [seed_data, experimentalist_result_1, experiment_runner_result_1, theorist_result_1, ... ]
... and then resolve the deltas whenever we need a particular code.


In [ ]:
@dataclass(frozen=True)
class State:
    df: Optional[pd.DataFrame]


@dataclass(frozen=True)
class StateDelta(State):
    kind: Literal["extend", "replace"]
    def __radd__(self, other: Union[State, StateDelta]):
        if self.kind == "replace":
            updates = dict()
            for f in dataclasses.fields(other):
                if (value := getattr(self, f.name)) is not None:
                    updates[f.name] = value
            return dataclasses.replace(other, **updates)
        elif self.kind == "extend":
            updates = dict()
            for f in dataclasses.fields(other):
                if (value := getattr(self, f.name)) is not None:
                    other_value = getattr(other, f.name)
                    updates[f.name] = pd.concat((other_value, value), ignore_index=True)
            return dataclasses.replace(other, **updates)

@dataclass(frozen=True)
class StateDeltaAltenative(State):
    kind: Literal["extend", "replace"]

    def __radd__(self, other: Union[State, StateDelta]):
        updates = dict()
        for f in dataclasses.fields(other):
            if (value := getattr(self, f.name)) is not None:
                if self.kind == "replace":
                    updates[f.name] = value
                elif self.kind == "extend":
                    other_value = getattr(other, f.name)
                    updates[f.name] = pd.concat((other_value, value), ignore_index=True)
        return dataclasses.replace(other, **updates)


In [ ]:
State(df=pd.DataFrame({"a": [1], "b": ["f"]})) + StateDeltaAltenative(kind="extend", df=pd.DataFrame({"a":[2], "b":["s"]}))

State(df=   a  b
0  1  f
1  2  s)

In [ ]:
State(df=pd.DataFrame({"a": [1], "b": ["f"]})) + StateDeltaAltenative(kind="replace", df=pd.DataFrame({"a":[2], "b":["s"]}))

State(df=   a  b
0  2  s)

In [ ]:
State(df=pd.DataFrame({"a": [1]})) \
+ StateDeltaAltenative(kind="replace", df=pd.DataFrame({"a":[2]})) \
+ StateDeltaAltenative(kind="extend", df=pd.DataFrame({"a":[3]})) \
+ StateDeltaAltenative(kind="extend", df=pd.DataFrame({"a":[4]}))

State(df=   a
0  2
1  3
2  4)

## Generic State and StateDelta

In [ ]:
class State:
    initial: Any
    deltas: List[StateDelta[State]]

S = TypeVar("S")

class StateDelta(S):
    value: S
    kind: Literal["replace", "extend"]


TypeError: TypeVar(name, constraint, ...): constraints must be types. Got (~S,).

In [ ]:
State = TypeVar("S")

class StateWithDeltas(Generic[State]):
    initial: Any
    deltas: List[StateDelta[State]]

class StateDelta(Generic[State]):
    value: State
    kind: Literal[]


SyntaxError: invalid syntax (2210526089.py, line 9)